In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tqdm
import random

from experiments.npe.model import get_npe_model
from experiments.npe_bt.simulate import show_simulation

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading chipmunk for Linux (64bit) [/home/shreyask/anaconda3/envs/tensorflow2/lib/python3.7/site-packages/pymunk/libchipmunk.so]


In [2]:
data = np.load('data/circles_uc1.npz', allow_pickle=True)

train_x = []
val_x = []

for i in range((data['config'].item()['num_objects'] - 1) * 2 + 1):
    train_x.append(data['arr_%i' % (i)])
    val_x.append(data['arr_%i' % (i + (data['config'].item()['num_objects'] - 1) * 2 + 1)])
    
train_y = data['train_y']
val_y = data['val_y']

In [ ]:
BATCH_SIZE = 50
ITERATIONS = 1000
BIG_BATCH = 131072

model_vanilla = get_npe_model(max_pairs = 2, probabilistic = True)
model_vanilla.compile(loss='msle', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0003))

model_uc = get_npe_model(max_pairs = 2, probabilistic = True)
model_uc.compile(loss='msle', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0003))

def sample_indexes(indexes):
    sampled_x = [
        x[indexes] for x in train_x
    ]
    
    sampled_y = train_y[indexes]
    
    return sampled_x, sampled_y


def sample_randomly():
    indexes = np.random.choice(len(train_x[0]), BATCH_SIZE)
    return sample_indexes(indexes)

def get_uncertainty(model, samples=100):
    xs = np.zeros((len(train_x[0]), samples))
    ys = np.zeros((len(train_x[0]), samples))
    
    for s in range(samples):
        p = model.predict(train_x, batch_size=BIG_BATCH)
        xs[:, s] = p[:, 0]
        ys[:, s] = p[:, 1]
        
    return (xs.std(axis=1) + ys.std(axis=1)) / 2

def sample_uncertainty(model, samples=100):
    var = get_uncertainty(model, samples=samples)
    indexes = np.argsort(var)[::-1]
    
    return sample_indexes(indexes[:BATCH_SIZE])

def join_xs(pool, new):
    return [
        np.concatenate((a, b))
        for a, b in zip(pool, new)
    ]

def join_ys(pool, new):
    return np.concatenate((pool, new))


pool_vx, pool_vy = sample_randomly()
pool_ux, pool_uy = sample_randomly()

vanilla = []
uc = []

for iteration in range(ITERATIONS):
    hv = model_vanilla.fit(pool_vx, pool_vy, epochs = 1, verbose = 0, batch_size = BATCH_SIZE, shuffle = True)
    hu = model_uc.fit(pool_ux, pool_uy, epochs = 1, verbose = 0, batch_size = BATCH_SIZE, shuffle = True)
    
    val_v = model_vanilla.evaluate(val_x, val_y, batch_size=BIG_BATCH, verbose=0)
    val_u = model_uc.evaluate(val_x, val_y, batch_size=BIG_BATCH, verbose=0)
    
    #print(val_v, val_u)
    
    print("Iteration %i, Vanilla: %f, UC: %f, (%i)" % (
        iteration,
        val_v,
        val_u,
        pool_ux[0].shape[0]
    ))
    
    vx, vy = sample_randomly()
    pool_vx, pool_vy = join_xs(pool_vx, vx), join_ys(pool_vy, vy)
    
    if iteration < 40:
        ux, uy = sample_randomly()
    else:
        ux, uy = sample_uncertainty(model_uc)

    pool_ux, pool_uy = join_xs(pool_ux, ux), join_ys(pool_uy, uy)
    
    vanilla.append(val_v)
    uc.append(val_u)

Iteration 0, Vanilla: 0.025608, UC: 0.026390, (50)
Iteration 1, Vanilla: 0.025373, UC: 0.025703, (100)
Iteration 2, Vanilla: 0.025314, UC: 0.025129, (150)
Iteration 3, Vanilla: 0.025328, UC: 0.024758, (200)
Iteration 4, Vanilla: 0.025325, UC: 0.024663, (250)
Iteration 5, Vanilla: 0.025358, UC: 0.024524, (300)
Iteration 6, Vanilla: 0.025264, UC: 0.024303, (350)
Iteration 7, Vanilla: 0.025239, UC: 0.024237, (400)
Iteration 8, Vanilla: 0.024881, UC: 0.024327, (450)
Iteration 9, Vanilla: 0.024625, UC: 0.023810, (500)
Iteration 10, Vanilla: 0.024372, UC: 0.023763, (550)
Iteration 11, Vanilla: 0.024115, UC: 0.023567, (600)
Iteration 12, Vanilla: 0.023956, UC: 0.023678, (650)
Iteration 13, Vanilla: 0.023812, UC: 0.023333, (700)
Iteration 14, Vanilla: 0.023705, UC: 0.023188, (750)
Iteration 15, Vanilla: 0.023546, UC: 0.023223, (800)
Iteration 16, Vanilla: 0.023406, UC: 0.023075, (850)
Iteration 17, Vanilla: 0.023169, UC: 0.022942, (900)
Iteration 18, Vanilla: 0.023049, UC: 0.022759, (950)
Iter

In [67]:
pool_vy.shape

(1600, 2)

In [68]:
pool_vy

array([[ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ],
       [ 0.        , -0.94691892],
       [ 0.        ,  0.        ]])

In [70]:
pool_uy.shape

(1600, 2)